In [16]:
# source https://medium.com/analytics-vidhya/finding-nearest-pair-of-latitude-and-longitude-match-using-python-ce50d62af546

from math import radians, cos, sin, asin, sqrt
import pandas as pd
import io
import sqlalchemy as sa
import urllib
import numpy as np
from sklearn.neighbors import BallTree

In [2]:
class LatLonMatch:
    def haversine(self, lon1, lat1, lon2, lat2):
        """
        Calculate the great circle distance between two points 
        on the earth (specified in decimal degrees)
        """
        # convert decimal degrees to radians
        lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
        # haversine formula
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
        c = 2 * asin(sqrt(a))
        # Radius of earth in kilometers is 6371
        km = 6371 * c
        return km

    def find_nearest(self, lat, long, dfName, colName='Nazwa'):
        distances = dfName.apply(
            lambda row: self.dist(lat, long, row['lat'], row['lon']),
            axis=1)
        return dfName.loc[distances.idxmin(), colName]

    def dist(self, lat1, long1, lat2, long2):
        lat1, long1, lat2, long2 = map(radians, [lat1, long1, lat2, long2])
        # haversine formula
        dlon = long2 - long1
        dlat = lat2 - lat1
        a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
        c = 2 * asin(sqrt(a))
        # Radius of earth in kilometers is 6371
        km = 6371 * c
        return km

In [4]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

SERVER = os.environ.get("SERVER")
DATABASE = os.environ.get("DATABASE")

USERNAME = os.environ.get("USERNAME")
PASSWORD = os.environ.get("PASSWORD")

conn = urllib.parse.quote_plus('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+str(
    SERVER)+';DATABASE='+str(DATABASE)+';UID='+str(USERNAME)+';PWD=' + str(PASSWORD))
engine = sa.create_engine(
    'mssql+pyodbc:///?odbc_connect={}'.format(conn), fast_executemany=True)
query_synop = """
/****** Script for SelectTopNRows command from SSMS  ******/
SELECT  [IDStacji]
      ,[Nazwa]
      ,[Kraj]
      ,[Szerokosc]
      ,[Dlugosc]
  FROM [Synop].[dbo].[Stacje]
  where Kraj in ('Poland')
"""

SynopStacje = pd.read_sql(query_synop, engine)
SynopStacje = SynopStacje.rename(
    columns={'Szerokosc': 'lat', 'Dlugosc': 'lon'})
SynopStacje = SynopStacje.astype({'lat': 'float', 'lon': 'float'})
query_smog = """
SELECT  
      [KodStacji]
      ,[Latitude]
      ,[Longitude]
  FROM [SmogoliczkaArchive].[dbo].[stacje_v2]
"""
SmogoliczkaStacje = pd.read_sql(query_smog, engine)
SmogoliczkaStacje = SmogoliczkaStacje.rename(
    columns={'Latitude': 'lat', 'Longitude': 'lon'})

In [5]:
SmogoliczkaStacje['Nazwa'] = SmogoliczkaStacje.apply(lambda row: LatLonMatch().find_nearest(row['lat'], row['lon'],SynopStacje,colName='Nazwa'), axis=1)
# To check the data frame if it has a new column of hotel name (for each and every member's location in the list)
SmogoliczkaStacje = pd.merge(SmogoliczkaStacje,SynopStacje,left_on='Nazwa', right_on='Nazwa', how='left')
SmogoliczkaStacje=SmogoliczkaStacje.rename(columns = {'lat_x':'smog_lat','lon_x':'smog_lon','lat_y':'synop_lat','lon_y':'synop_lon'})
SmogoliczkaStacje['distance'] = [LatLonMatch().haversine(SmogoliczkaStacje.smog_lon[i],SmogoliczkaStacje.smog_lat[i],SmogoliczkaStacje.synop_lon[i],SmogoliczkaStacje.synop_lat[i]) for i in range(len(SmogoliczkaStacje))]
SmogoliczkaStacje['distance'] = SmogoliczkaStacje['distance'].round(decimals=3)

In [6]:
SmogoliczkaStacje.tail()


,KodStacji,smog_lat,smog_lon,Nazwa,IDStacji,Kraj,synop_lat,synop_lon,distance
1031,ZpSzczPils02,53.432169,14.553900,Szczecin,12205,Poland,53.397047,14.620797,5.908
1032,ZpSzczWSSEEnerg,53.420475,14.561934,Szczecin,12205,Poland,53.397047,14.620797,4.691
1033,ZpSzczWSSESped6,53.415043,14.555347,Szczecin,12205,Poland,53.397047,14.620797,4.778
1034,ZpWalWalczWSSE,53.263667,16.492596,Piła,12230,Poland,53.130258,16.747900,22.567
1035,ZpWiduBulRyb,53.122325,14.382245,Szczecin,12205,Poland,53.397047,14.620797,34.423


In [7]:
SmogoliczkaStacje.to_sql("synop_smog", engine, if_exists="replace",index=False)

In [3]:
######## Przypisywanie najbliższych stacji meteo do Smogoliczki 
conn= urllib.parse.quote_plus('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+str(server)+';DATABASE='+str(database)+';UID='+str(username)+';PWD='+ str(password))
engine = sa.create_engine('mssql+pyodbc:///?odbc_connect={}'.format(conn),fast_executemany=True)
query_synop = """
/****** Script for SelectTopNRows command from SSMS  ******/
SELECT  [IDStacji]
      ,[Nazwa]
      ,[Kraj]
      ,[Szerokosc]
      ,[Dlugosc]
  FROM [Synop].[dbo].[Stacje]
  where Kraj in ('Poland')
"""

SynopStacje = pd.read_sql(query_synop, engine)
SynopStacje=SynopStacje.rename(columns = {'Szerokosc':'lat','Dlugosc':'lon'})
SynopStacje = SynopStacje.astype({'lat': 'float', 'lon': 'float'})

In [4]:
query_smog = """
SELECT  
      [KodStacji]
      ,[Latitude]
      ,[Longitude]
  FROM [SmogoliczkaArchive].[dbo].[stacje_v2]
"""
SmogoliczkaStacje = pd.read_sql(query_smog, engine)
SmogoliczkaStacje=SmogoliczkaStacje.rename(columns = {'Latitude':'lat','Longitude':'lon'})


In [5]:
SmogoliczkaStacje.head()

,KodStacji,lat,lon
0,DsBialka,51.197783,16.117390
1,DsBielGrot,50.682510,16.617348
2,DsBogatFrancMOB,50.940998,14.916790
3,DsBogChop,50.905856,14.967175
4,DsBogZatonieMob,50.943245,14.913327


In [6]:
SynopStacje.head()

,IDStacji,Nazwa,Kraj,lat,lon
0,12001,Petrobaltic Beta,Poland,55.466667,18.166667
1,12100,Kołobrzeg,Poland,54.182823,15.580516
2,12105,Koszalin,Poland,54.204928,16.156022
3,12106,Koszalin Zegrze Pom.,Poland,54.042222,16.263611
4,12115,Ustka,Poland,54.586395,16.854320


In [12]:
def degrees_to_radians(df):
    df['lat_rad'] = np.deg2rad(df['lat'])
    df['lon_rad'] = np.deg2rad(df['lon'])
    return df


def radians_to_kilometers(rad):
    # Options here: https://geopy.readthedocs.io/en/stable/#module-geopy.distance
    earth_radius = 6371.0087714150598
    return rad * earth_radius

In [17]:
SmogoliczkaStacje=degrees_to_radians(SmogoliczkaStacje)
SynopStacje=degrees_to_radians(SynopStacje)

In [18]:
ball = BallTree(SynopStacje[["lat_rad", "lon_rad"]], metric='haversine')
k = 3
distances, indices = ball.query(SmogoliczkaStacje[["lat_rad", "lon_rad"]], k = k)

In [19]:
distances=radians_to_kilometers(distances)

In [20]:
columns=['distance_1', 'distance_2','distance_3']
distance_df=pd.DataFrame(distances,columns=columns)
columns=['station_1', 'station_2','station_3']
indices_df=pd.DataFrame(indices,columns=columns)
dict_loc_a=SynopStacje['Nazwa'].to_dict()
indices_df=indices_df.replace({"station_1": dict_loc_a,'station_2':dict_loc_a,'station_3':dict_loc_a})
test=pd.concat([SmogoliczkaStacje, indices_df,distance_df], axis=1)

In [21]:
test.head()

,KodStacji,lat,lon,lat_rad,lon_rad,station_1,station_2,station_3,distance_1,distance_2,distance_3
0,DsBialka,51.197783,16.117390,0.893570,0.281302,Legnica,Jelenia Góra,Wrocław-Strachowice,6.313416,40.281631,54.106779
1,DsBielGrot,50.682510,16.617348,0.884577,0.290027,Kłodzko,Wrocław-Strachowice,Wrocław,27.273630,50.928575,55.549745
2,DsBogatFrancMOB,50.940998,14.916790,0.889088,0.260347,Zgorzelec,Śnieżka,Jelenia Góra,22.886553,59.510855,61.247268
3,DsBogChop,50.905856,14.967175,0.888475,0.261226,Zgorzelec,Śnieżka,Jelenia Góra,25.714147,55.030151,57.574262
4,DsBogZatonieMob,50.943245,14.913327,0.889127,0.260287,Zgorzelec,Śnieżka,Jelenia Góra,22.741395,59.817677,61.506697


In [27]:
test['station_1_id']=test['station_1'].replace(SynopStacje.set_index("Nazwa")['IDStacji'].to_dict())
test['station_2_id']=test['station_2'].replace(SynopStacje.set_index("Nazwa")['IDStacji'].to_dict())
test['station_3_id']=test['station_3'].replace(SynopStacje.set_index("Nazwa")['IDStacji'].to_dict())

In [28]:
test.head()

,KodStacji,lat,lon,lat_rad,lon_rad,station_1,station_2,station_3,distance_1,distance_2,distance_3,station_1_id,station_2_id,station_3_id
0,DsBialka,51.197783,16.117390,0.893570,0.281302,Legnica,Jelenia Góra,Wrocław-Strachowice,6.313416,40.281631,54.106779,12415,12500,12424
1,DsBielGrot,50.682510,16.617348,0.884577,0.290027,Kłodzko,Wrocław-Strachowice,Wrocław,27.273630,50.928575,55.549745,12520,12424,12425
2,DsBogatFrancMOB,50.940998,14.916790,0.889088,0.260347,Zgorzelec,Śnieżka,Jelenia Góra,22.886553,59.510855,61.247268,12405,12510,12500
3,DsBogChop,50.905856,14.967175,0.888475,0.261226,Zgorzelec,Śnieżka,Jelenia Góra,25.714147,55.030151,57.574262,12405,12510,12500
4,DsBogZatonieMob,50.943245,14.913327,0.889127,0.260287,Zgorzelec,Śnieżka,Jelenia Góra,22.741395,59.817677,61.506697,12405,12510,12500


In [29]:
columns = ['KodStacji',
 'lat',
 'lon',
 'lat_rad',
 'lon_rad',
 'station_1_id',           
 'station_1',
 'distance_1',
'station_2_id',
'station_2',
'distance_2',
 'station_3',
 'distance_3',
 'station_3_id']

In [30]:
test = test[columns]

In [31]:
test.drop(columns=['lat_rad','lon_rad'],inplace=True)

C:\Users\p_sierkin\Miniconda3\envs\real_estate\lib\site-packages\pandas\core\frame.py:3930: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [32]:
test.head()

,KodStacji,lat,lon,station_1_id,station_1,distance_1,station_2_id,station_2,distance_2,station_3,distance_3,station_3_id
0,DsBialka,51.197783,16.117390,12415,Legnica,6.313416,12500,Jelenia Góra,40.281631,Wrocław-Strachowice,54.106779,12424
1,DsBielGrot,50.682510,16.617348,12520,Kłodzko,27.273630,12424,Wrocław-Strachowice,50.928575,Wrocław,55.549745,12425
2,DsBogatFrancMOB,50.940998,14.916790,12405,Zgorzelec,22.886553,12510,Śnieżka,59.510855,Jelenia Góra,61.247268,12500
3,DsBogChop,50.905856,14.967175,12405,Zgorzelec,25.714147,12510,Śnieżka,55.030151,Jelenia Góra,57.574262,12500
4,DsBogZatonieMob,50.943245,14.913327,12405,Zgorzelec,22.741395,12510,Śnieżka,59.817677,Jelenia Góra,61.506697,12500


In [215]:
test[test['KodStacji']=='DsBogChop']

,KodStacji,lat,lon,station_1_id,station_1,distance_1,station_2_id,station_2,distance_2,station_3,distance_3,station_3_id
3,DsBogChop,50.905856,14.967175,12405,Zgorzelec,25.714147,12510,Śnieżka,55.030151,Jelenia Góra,57.574262,12500


In [33]:
test.to_sql("synop_smogv2", engine, if_exists="replace",index=False)